In [25]:
import polars as pl
from typing import List, Tuple, Union, Optional, Callable, Any, Dict
from datetime import datetime
import uuid
from llama_cpp import Llama


In [26]:
llm = Llama(model_path="/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf",chat_format="chatml",n_ctx=0,n_gpu_layers = -1)

llama_model_loader: loaded meta data with 25 key-value pairs and 995 tensors from /Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ehartford_dolphin-2.5-mixtral-8x7b
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dim

In [27]:
#the schema of the df is: 
messages = [
          {"role": "system", "content": "You are an assistant who perfectly describes python packages. GANG GANG"},
          {
              "role": "user",
              "content": "What is polars library in python?"
          }
      ]
results = llm.create_chat_completion(
      messages = messages,
)



llama_print_timings:        load time =    2357.07 ms
llama_print_timings:      sample time =       5.72 ms /    80 runs   (    0.07 ms per token, 13995.80 tokens per second)
llama_print_timings: prompt eval time =    2356.95 ms /    37 tokens (   63.70 ms per token,    15.70 tokens per second)
llama_print_timings:        eval time =    3434.93 ms /    79 runs   (   43.48 ms per token,    23.00 tokens per second)
llama_print_timings:       total time =    5886.74 ms


In [4]:
len(tokenizer.encode(str(messages)))

44

In [28]:

class ChatFrame:
      chat_schema = {"role":pl.Utf8,
                   "content":pl.Utf8,
                   "time_stamps":pl.Datetime,
                   "author":pl.Utf8,
                   "message_id":pl.Utf8,
                   "chat_id":pl.Utf8, 
                   "author":pl.Utf8,
                   "prompt": pl.List(pl.Struct({"role":pl.Utf8,"content":pl.Utf8})),
                   'prompt_tokens': pl.Int64, 
                   'completion_tokens': pl.Int64, 
                   'total_tokens': pl.Int64
}     
      chat_ml_schema =  {"chat_ml":pl.Struct({"role":pl.Utf8,"content":pl.Utf8}),
                   "time_stamps":pl.Datetime,
                   "author":pl.Utf8,
                   "message_id":pl.Utf8,
                   "chat_id":pl.Utf8, 
                   'completion_tokens': pl.Int64, 
}
      chat_nested_schema = {"role":pl.Utf8,
                   "content":pl.Utf8,
                   "time_stamps":pl.Datetime,
                   "author":pl.Utf8,
                   "message_id":pl.Utf8,
                   "chat_id":pl.Utf8, 
                   "prompt": pl.List(pl.Struct({"role":pl.Utf8,"content":pl.Utf8})),
                  "usage": pl.Struct({'prompt_tokens': pl.Int64, 'completion_tokens': pl.Int64, 'total_tokens': pl.Int64})
}
      def __init__(self,llm: Llama):
            self.chat_id = str(uuid.uuid4())
            # self.messages = chat_ml_messages
            # self.time_stamps = self.validate_timestamps(time_stamps)
            self.df = pl.DataFrame(schema=self.chat_schema)
            self.llm = llm
                        
      
      def add_from_chat_ml(self,chat_ml_messages: List[Dict[str, str]] , 
                           time_stamps: Optional[List[datetime]]=None,
                           author: Optional[List[str]]=None,):
            if time_stamps is None:
                  time_stamps = [datetime.now() for _ in range(len(chat_ml_messages))]

            message_id = ["chatml-" + str(uuid.uuid4()) for _ in range(len(chat_ml_messages))]
            if author is None:
                  author = ["import" for _ in range(len(chat_ml_messages))]
            chat_id = [self.chat_id for _ in range(len(chat_ml_messages))]
            
            message_tokens_len = [len(self.llm.tokenize(bytes(message["role"]+ message["content"], "utf-8"))) for message in chat_ml_messages]
            

            nested_df = pl.DataFrame({"chat_ml":chat_ml_messages,
                                      "time_stamps":time_stamps,
                                      "author":author,
                                      "message_id":message_id,
                                      "chat_id":chat_id,
                                      "completion_tokens":message_tokens_len,
                                    }
                                      ,schema=self.chat_ml_schema)
            # createa  list of ["chatml-" + str(uuid.uuid4())] with the same length as the number of messages
            
            message_df = nested_df.unnest("chat_ml")
            self.df = pl.concat([chat.df,message_df],how="diagonal",)
            return message_df                 
      
      def add_message_from_results_dict(self, results_dict: dict, prompt: List[Dict[str, str]]):
            
            data = {"role":[results_dict["choices"][0]["message"]["role"]],
                        "content":[results_dict["choices"][0]["message"]["content"]],
                        "author":[results_dict["model"]],
                        "message_id":[results_dict["id"]],
                        "chat_id":self.chat_id,
                        "prompt":[prompt],
                        "usage":[results_dict["usage"]],
                        "time_stamps":[datetime.fromtimestamp(results_dict["created"])],
            
                        }
            results_df = pl.DataFrame(data=data,schema=self.chat_nested_schema).unnest("usage")
            self.df = pl.concat([self.df,results_df],how="vertical")
            return results_df
      
      def to_chat_ml(self):
            return self.df.select(pl.struct(["role","content"]).alias("chat_ml")).to_dict(as_series=False)["chat_ml"]
      
      def reply(self, message: str):
            user_as_chat_ml = {"role":"user","content":message}
            self.add_from_chat_ml([user_as_chat_ml])
            prompt = self.to_chat_ml()
            results = self.llm.create_chat_completion(
                  messages = prompt,
            )
            self.add_message_from_results_dict(results, prompt=prompt)
            return results

            
chat = ChatFrame(llm)
message_df = chat.add_from_chat_ml(messages)
results_df = chat.add_message_from_results_dict(results,prompt=messages)
chat.to_chat_ml()



[{'role': 'system',
  'content': 'You are an assistant who perfectly describes python packages. GANG GANG'},
 {'role': 'user', 'content': 'What is polars library in python?'},
 {'role': 'assistant',
  'content': ' Polars is a Python package that provides a fast, efficient, and easy-to-use DataFrame implementation. It is designed to handle large datasets efficiently by leveraging the power of Rust programming language under the hood. Polars offers similar functionality as pandas but with better performance and memory usage. The library supports various operations like filtering, groupby, joins, and more.'}]

In [29]:
chat.df

role,content,time_stamps,author,message_id,chat_id,prompt,prompt_tokens,completion_tokens,total_tokens
str,str,datetime[μs],str,str,str,list[struct[2]],i64,i64,i64
"""system""","""You are an ass…",2024-01-09 01:26:54.497928,"""import""","""chatml-0ab38b2…","""dab3e3a6-af3b-…",null,null,16,null
"""user""","""What is polars…",2024-01-09 01:26:54.497932,"""import""","""chatml-c2ff0f0…","""dab3e3a6-af3b-…",null,null,10,null
"""assistant""",""" Polars is a P…",2024-01-09 01:26:35,"""/Users/tommaso…","""chatcmpl-b8ef6…","""dab3e3a6-af3b-…","[{""system"",""You are an assistant who perfectly describes python packages. GANG GANG""}, {""user"",""What is polars library in python?""}]",37,79,116


In [30]:
chat.reply("And how do I concatenate multiple columns into a single column row?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    2357.07 ms
llama_print_timings:      sample time =      19.39 ms /   240 runs   (    0.08 ms per token, 12378.15 tokens per second)
llama_print_timings: prompt eval time =    2337.53 ms /    26 tokens (   89.90 ms per token,    11.12 tokens per second)
llama_print_timings:        eval time =   10504.38 ms /   239 runs   (   43.95 ms per token,    22.75 tokens per second)
llama_print_timings:       total time =   13149.42 ms


{'id': 'chatcmpl-96e10731-9dda-4434-bf44-936934fc23d3',
 'object': 'chat.completion',
 'created': 1704760022,
 'model': '/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "To concatenate multiple columns into a single column row in Polars, you can use the `concat_str()` function. Here's an example:\n\n```python\nimport polars as pl\n\n# Create a sample DataFrame\ndf = pl.DataFrame({\n    'name': ['Alice', 'Bob', 'Charlie'],\n    'age': [25, 30, 35],\n    'country': ['USA', 'Canada', 'Mexico']\n})\n\n# Concatenate multiple columns into a single column row\ndf_concat = df.select(pl.concat_str(['name', 'age', 'country']))\n\nprint(df_concat)\n```\n\nOutput:\n\n```\nshape: (3, 1)\n┌─────┐\n│ name │\n│ ---  │\n├─────┤\n│ Alice25USA │\n│ Bob30Canada │\n│ Charlie35Mexico │\n└─────┘\n```"},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens':

In [20]:
chat.reply("what about if I want to do it for multiple rows?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    3988.32 ms
llama_print_timings:      sample time =      30.91 ms /   381 runs   (    0.08 ms per token, 12326.91 tokens per second)
llama_print_timings: prompt eval time =    1081.92 ms /    23 tokens (   47.04 ms per token,    21.26 tokens per second)
llama_print_timings:        eval time =   44135.18 ms /   380 runs   (  116.15 ms per token,     8.61 tokens per second)
llama_print_timings:       total time =   45751.34 ms


{'id': 'chatcmpl-f9c4160c-98d5-4a92-949f-3a097a674f75',
 'object': 'chat.completion',
 'created': 1704759076,
 'model': '/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'To concatenate multiple columns into a single column row for multiple rows, you can use the same `concat_str` function as before. Here\'s an example:\n\n```python\nimport polars as pl\n\n# Create a sample DataFrame with more rows\ndf = pl.DataFrame({\n    "name": ["Alice", "Bob", "Charlie", "David"],\n    "age": [25, 30, 35, 40],\n    "city": ["New York", "San Francisco", "Los Angeles", "Chicago"]\n})\n\n# Concatenate multiple columns into a single column row for multiple rows\ndf_concat = df.select(pl.concat_str(["name", "age", "city"], separator="_").alias("full_info"))\n\nprint(df_concat)\n```\n\nOutput:\n\n```\nshape: (4, 1)\n┌─────────────┐\n│ full_info     │\n│ ---

In [ ]:
pl.concat([chat.df,message_df],how="diagonal",)

role,content,time_stamps,author,message_id,chat_id,prompt:,prompt_tokens,completion_tokens,total_tokens
enum,str,datetime[μs],str,str,str,list[struct[2]],i64,i64,i64
"""system""","""You are an ass…",2024-01-09 00:11:42.046525,"""import""","""chatml-a94266f…","""23d35fdf-793c-…",null,null,10,null
"""user""","""What is polars…",2024-01-09 00:11:42.046527,"""import""","""chatml-af003c5…","""23d35fdf-793c-…",null,null,8,null


In [ ]:
chat.df.select(pl.struct([pl.col("role"),pl.col("content")]).alias("chat_ml")).to_dict(as_series=False)

{'chat_ml': [{'role': 'system',
   'content': 'You are an assistant who perfectly describes python packages.'},
  {'role': 'user', 'content': 'What is polars library in python?'}]}

In [ ]:
id = str(uuid.uuid4())
print(type(id))
pl.lit(id)

<class 'str'>


<Expr ['String(c7a38370-a6d0-4f64-8783…'] at 0x10BC09430>

In [ ]:
# df_single = pl.DataFrame(data={"chat_id": [1]})
df_single = df.with_columns(pl.lit(1).alias("chat_id"))
df_single

role,content,time_stamps,chat_id
enum,str,datetime[μs],i32
"""system""","""You are an ass…",2024-01-08 20:46:11.375016,1
"""user""","""What is polars…",2024-01-08 20:46:11.375020,1


In [ ]:
from llama_cpp import Llama


Llama.generate: prefix-match hit

llama_print_timings:        load time =    3724.28 ms
llama_print_timings:      sample time =       8.34 ms /   103 runs   (    0.08 ms per token, 12347.16 tokens per second)
llama_print_timings: prompt eval time =    1077.72 ms /    23 tokens (   46.86 ms per token,    21.34 tokens per second)
llama_print_timings:        eval time =   12324.44 ms /   102 runs   (  120.83 ms per token,     8.28 tokens per second)
llama_print_timings:       total time =   13537.53 ms


In [ ]:
results

{'id': 'chatcmpl-2e9e5958-e306-43f1-b92c-d304c817da98',
 'object': 'chat.completion',
 'created': 1704751139,
 'model': '/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': " Polars is a fast, efficient, and easy-to-use data processing library for Python. It provides high-performance DataFrame operations, similar to pandas, but with optimized memory usage and faster execution times. Polars is built on top of the Rust programming language, which allows it to leverage the power of Rust's performance and safety features. The library includes support for reading and writing various file formats, handling missing data, groupby operations, joins, and more."},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 37, 'completion_tokens': 102, 'total_tokens': 139}}

In [ ]:
{'id': 'chatcmpl-eab5813e-3c8a-474b-bbee-636b9385d955',
 'object': 'chat.completion',
 'created': 1704751049,
 'model': '/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': ' The Polars library in Python is a high-performance DataFrame library that provides fast and efficient data processing capabilities. It is designed to handle large datasets with ease, offering features such as lazy evaluation, parallel execution, and memory optimization. Polars supports various data types, including integers, floats, strings, booleans, and timestamps, making it suitable for a wide range of applications in data science, machine learning, and analytics.'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 33, 'completion_tokens': 92, 'total_tokens': 125}}

In [ ]:
[results_dict["choices"][0]["message"]["content"]]

[" Polars is a fast, efficient, and easy-to-use data processing library for Python. It provides high-performance DataFrame operations, similar to pandas, but with optimized memory usage and faster execution times. Polars is built on top of the Rust programming language, which allows it to leverage the power of Rust's performance and safety features. The library includes support for reading and writing various file formats, handling missing data, groupby operations, joins, and more."]

In [ ]:
messages

[{'role': 'system',
  'content': 'You are an assistant who perfectly describes python packages.'},
 {'role': 'user', 'content': 'What is polars library in python?'}]

In [ ]:
messages_series = pl.Series(messages)
messages_series.


OrderedDict([('', Struct({'role': String, 'content': String}))])

In [ ]:
schema = {"prompt:": pl.List(pl.Struct({"role":pl.Utf8,"content":pl.Utf8})),
          "role":pl.Utf8,
            "content":pl.Utf8,
            "author":pl.Utf8,
            "message_id":pl.Utf8,}
data = {"prompt:": [messages_series],
        "role"}
df = pl.DataFrame(data=data,schema=schema)
df

prompt:
list[struct[2]]
"[{""system"",""You are an assistant who perfectly describes python packages.""}, {""user"",""What is polars library in python?""}]"


In [ ]:
chat_schema = {"role":pl.Utf8,
                   "content":pl.Utf8,
                   "author":pl.Utf8,
                   "message_id":pl.Utf8,
                   "chat_id":pl.Utf8, 
                   "author":pl.Utf8,
                   "prompt": pl.List(pl.Struct({"role":role_dtype,"content":pl.Utf8})),
                   "usage": pl.Struct({'prompt_tokens': pl.Int64, 'completion_tokens': pl.Int64, 'total_tokens': pl.Int64})
}
def add_message_from_results_dict(self, results_dict: dict, prompt: List[Dict[str, str]]):
    data = {"role":[results_dict["choices"][0]["message"]["role"]],
            "content":[results_dict["choices"][0]["message"]["content"]],
            "author":[results_dict["model"]],
            "message_id":[results_dict["id"]],
            "chat_id":self.chat_id,
            "prompt":[prompt],
            "usage":[results_dict["usage"]]
    
            }
    df = pl.DataFrame(data=data,schema=chat_schema)
    self.df = pl.concat([self.df,df],how="vertical")


role,content,author,message_id,chat_id,prompt,usage
str,str,str,str,str,list[struct[2]],struct[3]
"""assistant""",""" Polars is a f…","""/Users/tommaso…","""chatcmpl-2e9e5…",null,"[{""system"",""You are an assistant who perfectly describes python packages.""}, {""user"",""What is polars library in python?""}]","{37,102,139}"
